In [ ]:
!pip install pandas==1.1.2

     |████████████████████████████████| 10.5MB 7.9MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.2 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

% cd '/content/drive/My Drive/cricket-analysis'

Mounted at /content/drive
/content/drive/My Drive/cricket-analysis


In [ ]:
!ls -lath

total 103K
-rw------- 1 root root 33K Sep 26 21:21 kohli.pkl
-rw------- 1 root root 29K Sep 26 21:21 sharma.pkl
-rw------- 1 root root 22K Sep 26 02:22 kohli_base.pkl
-rw------- 1 root root 19K Sep 26 02:22 sharma_base.pkl


In [ ]:
import pandas as pd
import numpy as np

from pickle import Unpickler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# with open('sharma.pkl', 'rb') as fp:
#   df_sharma = Unpickler(fp).load()

df_sharma = pd.read_pickle('sharma.pkl')
df_kohli = pd.read_pickle('kohli.pkl')

In [ ]:
print(f"sharma partnerships: {df_sharma.shape}")
print(f"kohli partnerships: {df_kohli.shape}")

sharma partnerships: (417, 8)
kohli partnerships: (477, 8)


In [ ]:
m1 = set(df_sharma[df_sharma['partner'] == 'V Kohli']['dom'].unique())
len(m1)

m2 = set(df_kohli[df_kohli['partner'] == 'RG Sharma']['dom'].unique())
len(m2)

In [ ]:
df_sharma.head()

,partner,total_runs,total_deliveries,runs_sharma,dom,result,home_venue,win_toss
0,S Dhawan,136,150,64,2017-06-04,1,0,0
1,V Kohli,56,73,27,2017-06-04,1,0,0
2,S Dhawan,138,151,78,2017-06-08,0,0,0
3,S Dhawan,23,33,12,2017-06-11,1,0,1
4,S Dhawan,87,88,41,2017-06-15,1,0,1


In [ ]:
df_kohli.head()

,partner,total_runs,total_deliveries,runs_kohli,dom,result,home_venue,win_toss
0,RG Sharma,56,73,29,2017-06-04,1,0,0
1,Yuvraj Singh,93,59,38,2017-06-04,1,0,0
2,HH Pandya,34,10,14,2017-06-04,1,0,0
3,S Dhawan,1,6,0,2017-06-08,0,0,0
4,S Dhawan,128,150,57,2017-06-11,1,0,1


In [ ]:
df_kohli[df_kohli['partner'] == 'RG Sharma'].shape
df_sharma[df_sharma['partner'] == 'V Kohli'].shape

In [ ]:
# df_sharma['runs_sharma'].fillna(0, inplace=True)
# df_kohli['runs_kohli'].fillna(0, inplace=True)

In [ ]:
def null_cols(data):

    """
    This function takes a dataframe df and shows the % null values in each column

    """
    nulls = data.isna().mean()*100
    nulls = pd.DataFrame(nulls.sort_values(ascending = False))
    nulls = nulls.reset_index().rename(columns = {"index": "feature", 0: "% nulls"})
    return nulls

In [ ]:
null_cols(df_sharma)
print("============")
null_cols(df_kohli)

,feature,% nulls
0,win_toss,0.0
1,home_venue,0.0
2,result,0.0
3,dom,0.0
4,runs_sharma,0.0
5,total_deliveries,0.0
6,total_runs,0.0
7,partner,0.0


,feature,% nulls
0,win_toss,0.0
1,home_venue,0.0
2,result,0.0
3,dom,0.0
4,runs_kohli,0.0
5,total_deliveries,0.0
6,total_runs,0.0
7,partner,0.0


In [ ]:
df_sharma_toss1 = df_sharma[df_sharma['win_toss']==1]
df_sharma_toss2 = df_sharma[df_sharma['win_toss']==0]

In [ ]:
# the matches where India won the toss
agg_sharma1 = df_sharma_toss1.groupby('partner').agg(TLPsRuns=('total_runs', 'sum'),
                                             TLPsBalls=('total_deliveries', 'sum'),
                                             PsCount=('dom', 'count'),
                                             TLRunsSharma=('runs_sharma', 'sum'),
                                             WinPerc=('result', 'mean'),
                                             WinToss=('win_toss', 'mean'))

agg_sharma1['AvgPsRuns'] = agg_sharma1['TLPsRuns']/agg_sharma1['PsCount']
agg_sharma1['AvgPsRunsSharma'] = agg_sharma1['TLRunsSharma']/agg_sharma1['PsCount']
agg_sharma1['StrikeRate'] = agg_sharma1['TLPsRuns']/agg_sharma1['TLPsBalls']
agg_sharma1.sort_values(by=['TLPsRuns', 'AvgPsRuns'], ascending=[False, False], inplace=True)
agg_sharma1.head(10)

,TLPsRuns,TLPsBalls,PsCount,TLRunsSharma,WinPerc,WinToss,AvgPsRuns,AvgPsRunsSharma,StrikeRate
partner,,,,,,,,,
V Kohli,2250,2260,33,1092,0.666667,1,68.181818,33.090909,0.995575
S Dhawan,2091,2483,51,947,0.686275,1,41.000000,18.568627,0.842126
SK Raina,796,818,20,365,0.700000,1,39.800000,18.250000,0.973105
MS Dhoni,599,661,18,303,0.611111,1,33.277778,16.833333,0.906203
AT Rayudu,435,435,6,258,0.833333,1,72.500000,43.000000,1.000000
AM Rahane,410,550,12,183,0.583333,1,34.166667,15.250000,0.745455
KL Rahul,381,418,6,214,1.000000,1,63.500000,35.666667,0.911483
RA Jadeja,313,311,6,166,0.500000,1,52.166667,27.666667,1.006431
Yuvraj Singh,212,310,8,105,0.250000,1,26.500000,13.125000,0.683871


In [ ]:
# the matches where India lost the toss
agg_sharma2 = df_sharma_toss2.groupby('partner').agg(TLPsRuns=('total_runs', 'sum'),
                                             TLPsBalls=('total_deliveries', 'sum'),
                                             PsCount=('dom', 'count'),
                                             TLRunsSharma=('runs_sharma', 'sum'),
                                             WinPerc=('result', 'mean'),
                                             WinToss=('win_toss', 'mean'))

agg_sharma2['AvgPsRuns'] = agg_sharma2['TLPsRuns']/agg_sharma2['PsCount']
agg_sharma2['AvgPsRunsSharma'] = agg_sharma2['TLRunsSharma']/agg_sharma2['PsCount']
agg_sharma2['StrikeRate'] = agg_sharma2['TLPsRuns']/agg_sharma2['TLPsBalls']
agg_sharma2.sort_values(by=['TLPsRuns', 'AvgPsRuns'], ascending=[False, False], inplace=True)
agg_sharma2.head(10)

,TLPsRuns,TLPsBalls,PsCount,TLRunsSharma,WinPerc,WinToss,AvgPsRuns,AvgPsRunsSharma,StrikeRate
partner,,,,,,,,,
S Dhawan,2650,3022,55,1158,0.618182,0,48.181818,21.054545,0.876903
V Kohli,2619,2897,45,1324,0.577778,0,58.200000,29.422222,0.904039
MS Dhoni,815,861,12,479,0.583333,0,67.916667,39.916667,0.946574
KL Rahul,781,851,9,438,0.666667,0,86.777778,48.666667,0.917744
AM Rahane,727,844,12,377,0.500000,0,60.583333,31.416667,0.861374
SS Iyer,390,339,5,204,0.600000,0,78.000000,40.800000,1.150442
SK Raina,273,324,10,133,0.400000,0,27.300000,13.300000,0.842593
G Gambhir,253,295,10,113,0.600000,0,25.300000,11.300000,0.857627
Yuvraj Singh,237,274,12,94,0.500000,0,19.750000,7.833333,0.864964


In [ ]:
# df_sharma_copy = df_sharma.copy()

agg_sharma = df_sharma_copy.groupby('partner').agg(TLPsRuns=('total_runs', 'sum'),
                                             TLPsBalls=('total_deliveries', 'sum'),
                                             PsCount=('dom', 'count'),
                                             TLRunsSharma=('runs_sharma', 'sum'),
                                             WinPerc=('result', 'mean'),
                                             WinToss=('win_toss', 'mean'))

agg_sharma['AvgPsRuns'] = agg_sharma['TLPsRuns']/agg_sharma['PsCount']
agg_sharma['AvgPsRunsSharma'] = agg_sharma['TLRunsSharma']/agg_sharma['PsCount']
agg_sharma['StrikeRate'] = agg_sharma['TLPsRuns']/agg_sharma['TLPsBalls']
agg_sharma.sort_values(by=['TLPsRuns', 'AvgPsRuns'], ascending=[False, False], inplace=True)
agg_sharma.head(10)

,TLPsRuns,TLPsBalls,PsCount,TLRunsSharma,WinPerc,WinToss,AvgPsRuns,AvgPsRunsSharma,StrikeRate
partner,,,,,,,,,
V Kohli,4869,5157,78,2416,0.615385,0.423077,62.423077,30.974359,0.944154
S Dhawan,4741,5505,106,2105,0.650943,0.481132,44.726415,19.858491,0.861217
MS Dhoni,1414,1522,30,782,0.600000,0.600000,47.133333,26.066667,0.929041
KL Rahul,1162,1269,15,652,0.800000,0.400000,77.466667,43.466667,0.915682
AM Rahane,1137,1394,24,560,0.541667,0.500000,47.375000,23.333333,0.815638
SK Raina,1069,1142,30,498,0.600000,0.666667,35.633333,16.600000,0.936077
AT Rayudu,486,508,10,282,0.800000,0.600000,48.600000,28.200000,0.956693
Yuvraj Singh,449,584,20,199,0.400000,0.400000,22.450000,9.950000,0.768836
SS Iyer,390,339,5,204,0.600000,0.000000,78.000000,40.800000,1.150442


In [ ]:
df_kohli_copy = df_kohli.copy()
agg_kohli = df_kohli_copy.groupby('partner').agg(total_ps_runs=('total_runs', 'sum'),
                                             total_ps_deliveries=('total_deliveries', 'sum'),
                                             count_ps=('dom', 'count'),
                                             total_runs_kohli=('runs_kohli', 'sum'),
                                             win_perc=('result', 'mean'))

agg_kohli['avg_ps_runs'] = agg_kohli['total_ps_runs']/agg_kohli['count_ps']
agg_kohli['avg_runs_kohli'] = agg_kohli['total_runs_kohli']/agg_kohli['count_ps']
agg_kohli['strike_rate'] = agg_kohli['total_ps_runs']/agg_kohli['total_ps_deliveries']
agg_kohli.sort_values(by=['total_ps_runs', 'avg_ps_runs'], ascending=[False, False], inplace=True)
agg_kohli.head(10)

,total_ps_runs,total_ps_deliveries,count_ps,total_runs_kohli,win_perc,avg_ps_runs,avg_runs_kohli,strike_rate
partner,,,,,,,,
RG Sharma,4869,5157,78,2305,0.615385,62.423077,29.551282,0.944154
S Dhawan,3005,3173,50,1386,0.640000,60.100000,27.720000,0.947053
SK Raina,2223,2154,42,1103,0.666667,52.928571,26.261905,1.032033
G Gambhir,1963,2270,34,970,0.705882,57.735294,28.529412,0.864758
MS Dhoni,1834,2105,49,954,0.612245,37.428571,19.469388,0.871259
AM Rahane,1708,1922,31,914,0.677419,55.096774,29.483871,0.888658
Yuvraj Singh,1146,1265,29,545,0.655172,39.517241,18.793103,0.905929
AT Rayudu,869,962,16,460,0.750000,54.312500,28.750000,0.903326
KM Jadhav,698,652,14,353,0.857143,49.857143,25.214286,1.070552


In [ ]:
agg_kohli.sort_values(by=['count_ps'], inplace=True, ascending=False)
agg_kohli

,total_ps_runs,total_ps_deliveries,count_ps,total_runs_kohli,average_ps_runs,average_runs_kohli
partner,,,,,,
RG Sharma,4869,5157,78,2305,62.423077,29.551282
S Dhawan,3005,3173,50,1386,60.100000,27.720000
MS Dhoni,1834,2105,49,954,37.428571,19.469388
SK Raina,2223,2154,42,1103,52.928571,26.261905
G Gambhir,1963,2270,34,970,57.735294,28.529412
AM Rahane,1708,1922,31,914,55.096774,29.483871
Yuvraj Singh,1146,1265,29,545,39.517241,18.793103
AT Rayudu,869,962,16,460,54.312500,28.750000
KM Jadhav,698,652,14,353,49.857143,25.214286
